## Testing auto-sumarrize function : extracting data from website

In [1]:
import urllib3
urllib3.disable_warnings()
from bs4 import BeautifulSoup

from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.corpus import stopwords
from string import punctuation

from nltk.probability import FreqDist
from heapq import nlargest

from collections import defaultdict

In [2]:
url = "https://hackernoon.com/why-python-is-the-best-choice-for-fintech-products-64a2eae7b8be"
# url = "https://itnext.io/list-of-free-resources-to-learn-natural-language-processing-ce27231e79a2"

In [3]:
def getTextWaPo(url):
    http = urllib3.PoolManager()
    page = http.request('GET', url)
    soup = BeautifulSoup(page.data.decode('UTF-8'),'html.parser')    # 'lxml' parser
    text = ' '.join(map(lambda p: p.text, soup.find_all('article')))
    text.encode('ascii', errors='replace').replace(b"?",b" ")
    return text

In [4]:
def summarize(text,n):
    sents = sent_tokenize(text)
    
    assert n <= len(sents)       # fucntion fails if article contains 2 function and you have to make summary of 3 sentence.
    
    word_sent = word_tokenize(text.lower())
    _stopwords = set(stopwords.words('english') + list(punctuation))
    
    word_sent = [word for word in word_sent if word not in _stopwords]

    freq = FreqDist(word_sent)
    
    ranking = defaultdict(int)
    
    for i,sent in enumerate(sents):
        for w in word_tokenize(sent.lower()):
            if w in freq:
                ranking[i] += freq[w]
                
    sents_idx = nlargest(n, ranking, key=ranking.get)
    
    return [sents[j] for j in sorted(sents_idx)]

In [5]:
text = getTextWaPo(url)

In [6]:
summarize(text,4)

['It’s flexible, it is able to adapt to the new users’ needs\u200a—\u200ait’s exactly what millennials want.',
 'It’s quite simple: with the Python/Django stack, you can build an MVP quite fast, which increases your chances to find your product/market fit.The only way fintech will be able to compete and/or collaborate with traditional banking and finance is adapting to changes and customer’s demands, offering additional services and improvements according to customer’s wishes.',
 'Thanks to Python’s open libraries, you don’t have to build your tools from scratch and develop the product and analyze large amounts of data in the shortest amounts of time.',
 'It’s the go-to language for data analysis, which makes it attractive for non-technical fields like business, and the best programming language for financial analysis.Again, I’m not trying to say that Python is the only way to go.']